_Última modificación: 09 de noviembre de 2024_
# Práctica 3: Identificación de frases clave y resumen automático de texto
**Hernández Jiménez Erick Yael**: 2023630748.

Escuela Superior de Cómputo.

Ingeniería en inteligencia Artificial. 5BV1.

Tecnologías para el Procesamiento de Lenguaje Natural.



## Resumen
En este cuaderno se usará el procesamiento de cuerpos para realizar un resumen automático extractivo de 3 documentos tras normalizarlos. A continuación se enlistan las bibliotecas utilizadas.

In [1]:
import nltk     # Biblioteca con las herramientas utilizadas para la manipulación y procesamiento de los documentos
from nltk.tokenize import word_tokenize, sent_tokenize  # Submódulo para tokenizar
from nltk.tokenize import RegexpTokenizer  # Para generar el filtro
nltk.download('punkt_tab')
nltk.download('stopwords')  # Lista con stopwords en distintos idiomas
import math                 # Para operaciones matemáticas complejas
from rake_nltk import Rake  # Algoritmo Rake modificado

[nltk_data] Downloading package punkt_tab to /home/hjey/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /home/hjey/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Generación de cuerpo de documentos
Extrayendo las primeras 3 cartas del libro de Frankstein, desde el enlace [“https://www.gutenberg.org/ebooks/84”](“https://www.gutenberg.org/ebooks/84”), los textos se guardaron manualmente como archivos de texto en la carpeta docs como [Carta_1](./docs/Carta_1.txt), [Carta_2](./docs/Carta_2.txt) y [Carta_3](./docs/Carta_3.txt). A partir de estos, incluimos un enunciado de cada uno.

In [2]:
with open("docs/Carta_1.txt", 'r', encoding="utf-8") as file:
    carta_1: str = file.read()
with open("docs/Carta_2.txt", 'r', encoding="utf-8") as file:
    carta_2: str = file.read()
with open("docs/Carta_3.txt", 'r', encoding="utf-8") as file:
    carta_3: str = file.read()

Tokenizamos en enunciados para, luego, escoger un enunciado al azar de cada capítulo y así generar el documento

In [3]:
# Tokenizamos por enunciados
original: list[str] = sent_tokenize(carta_1)
original.extend(sent_tokenize(carta_2))
original.extend(sent_tokenize(carta_3))

corpus: str = ""
for enunciado in original:
     corpus += str(enunciado) + " "
print(corpus)

 To Mrs. Saville, England. St. Petersburgh, Dec. 11th, 17—. You will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings. I arrived here yesterday, and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking. I am already far north of London, and as I walk in the streets of Petersburgh, I feel a cold northern breeze play upon my cheeks, which braces my nerves and fills me with delight. Do you understand this feeling? This breeze, which has travelled from the regions towards which I am advancing, gives me a foretaste of those icy climes. Inspirited by this wind of promise, my daydreams become more fervent and vivid. I try in vain to be persuaded that the pole is the seat of frost and desolation; it ever presents itself to my imagination as the region of beauty and delight. There, Margaret, the sun is for ever visible, its broad disk just skirting t

Con esto, tenemos el corpus sobre el que trabajaremos a lo largo de la práctica

## Normalización de textos general
El flujo que se usará, y su justificación se explicará a continuación:
1. Conversión a minúsculas: para evitar redundancias en el contenido significativo del cuerpo
2. Elimininación de espacios, números, signos de puntuación y el caracter '—': Estos caracteres se encuentra en los 3 archivos originales, siendo de carácter visual para separar diálogos y contextos en las frases. No aporta contenido al proceso de ninguno de los algoritmos por lo que su eliminación reducirá el análisis. Cabe mencionar que el caracter '—' es disntinto de '-', siendo el último relevante para la generación del resumen debido a que cambia el significado de las palabras adyacentes, por lo que se mantiene en el cuerpo.

In [4]:
# Conversión a minúsculas
corpus: str = corpus.lower()

# Tokenizamos por enunciados
enunciados: list[str] = sent_tokenize(corpus)

# Eliminamos los caracteres que no nos interesen de cada documento sin perder la estructura u orden
for i, enunciado in enumerate(enunciados):
# Para cada enunciadp 'enunciado' con índice 'i' en los elementos enumerados de 'documentos'
        enunciados[i] = ''.join([char for char in enunciado if char.isalpha() or char == ' ' or char == '-'])
        # Al elemento [i] lo reasignamos como la unión con el caractér vacío '' por cada caracter 'char'
        # si 'char' es una letra o es un espacio o el guión '-'

# Tokenizamos por palabras
palabras_por_enunciado: list[list[str]] = []
for enunciado in enunciados:
    palabras_por_enunciado.append(word_tokenize(enunciado))

# Imprimimos los resultados
for enunciado, palabras in zip(enunciados, palabras_por_enunciado):
    print(enunciado)
    print(palabras,"\n")

 to mrs saville england
['to', 'mrs', 'saville', 'england'] 

st petersburgh dec th 
['st', 'petersburgh', 'dec', 'th'] 

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings
['you', 'will', 'rejoice', 'to', 'hear', 'that', 'no', 'disaster', 'has', 'accompanied', 'the', 'commencement', 'of', 'an', 'enterprise', 'which', 'you', 'have', 'regarded', 'with', 'such', 'evil', 'forebodings'] 

i arrived here yesterday and my first task is to assure my dear sister of my welfare and increasing confidence in the success of my undertaking
['i', 'arrived', 'here', 'yesterday', 'and', 'my', 'first', 'task', 'is', 'to', 'assure', 'my', 'dear', 'sister', 'of', 'my', 'welfare', 'and', 'increasing', 'confidence', 'in', 'the', 'success', 'of', 'my', 'undertaking'] 

i am already far north of london and as i walk in the streets of petersburgh i feel a cold northern breeze play upon my cheeks which braces my nerves a

## Resumen automático extractivo de texto

### TF-IDF - NLTK
Siguiendo el ejemplo de [TURING](https://www.turing.com/kb/5-powerful-text-summarization-techniques-in-python), aplicaremos el algoritmo TF-IDF:

In [5]:
# Crearemos clases que conserven estos datos para analizarlos posteriormente
class TF_IDF():
    r'''
    Clase que contiene el resumen por TF-IDF de una serie de tokens:
    '''
    def __init__(self, tokens, corpus, lang: str = "english"):
        r'''
        # __init__
        - tokens: Any = iterable con los tokens del documento sobre el que se aplicará el algoritmo TF-IDF
        - lang: str = nombre del lenguaje a partir del cual se eliminarán las stop-words que NLTK tiene por defecto 
        '''
        self.tokens = tokens
        self.corpus = corpus
        self.stopWords: set = nltk.corpus.stopwords.words(lang) # Definimos el conjunto de stopwords
        self.tablaFrec: dict = {}   # Inicializamos con un diccionario vacío
        self.pesoEnun: dict = {}    # Inicializamos con un diccionario vacío
        self.totalPesos: int = 0    # Número descriptivo del total de los pesos
        self.promedio: int = 0      # Número descriptivo con el promedio de los pesos
        self.resumen: list = []     # Lista vacía para almacenar las palabras más frecuentes que conforman al resumen
        self.relevancia: float = 0  # Número descriptivo que indica la relevancia del enunciado en el corpus
    
    def calcularTF(self) -> None:
        r'''
        # generarTablaFreq
        Genera la tabla de frecuencias normalizadas del documento
        '''
        # Conteo de frecuencias
        for palabra in self.tokens:        # Por cada palabra en los tokens...
            if palabra in self.stopWords:  # si la palabra se incluyen en las stopwords...
                continue                # las omitimos y continuamos
            else:                       # de lo contrario...
                if palabra in self.tablaFrec:      # si la palabra ya se incluye en la tabla de frecuencias...
                    self.tablaFrec[palabra] += 1   # aumentamos en 1 el contador
                else:                           # sino...
                    self.tablaFrec[palabra] = 1    # Empezamos el conteo en 1
        # Normalización de frecuencias
        for palabra in self.tablaFrec.keys():
            self.tablaFrec[palabra] /= len(self.tokens)

    def calcularIDF(self) -> None:
        r'''
        #calcularIDF
        Calcular el peso que tiene cada palabra en los enunciados con respecto a todos los documentos
        '''
        num_docs = len(self.corpus)
        for palabra in self.tablaFrec:
        # Por cada palabra en la tabla de frecuencias...
            doc_count = sum(1 for documento in self.corpus if palabra in documento)
            # Aumentamos en uno el contador de documentos que contengan a la palabra de la iteración
            self.pesoEnun[palabra] = math.log(num_docs / (1 + doc_count)) # Se agrega el 1 para suavizar el peso de palabras raras
            # Para luego calcular el valor correspondiente al IDF y asignarlo al diccionario con las palabras del corpus

    def imprimir_tabla(self, tabla: dict) -> None:
        r'''
        #imprimir_tabla
        Imprime la tabla indicada
        '''
        for key in tabla:
            print(f"'{key}':{tabla[key]}")
        print("\n")

    def calcular_TF_IDF(self) -> None:
        self.calcularTF()
        self.calcularIDF()
        tf_idf = {}
        self.relevancia = 0
        for palabra in self.tablaFrec:
            tf = self.tablaFrec[palabra]
            idf = self.pesoEnun.get(palabra, 0)
            tf_idf[palabra] = tf * idf  # Multiplicación de TF e IDF
            self.relevancia += tf * idf
        self.resumen = sorted(tf_idf.items(), key=lambda x: x[1], reverse=True)


In [6]:
# Aplicamos el algoritmo sobre los enunciados
lista_td_idf: list[TF_IDF] = []
for palabras, enunciado in zip(palabras_por_enunciado, enunciados):
    lista_td_idf.append(TF_IDF(palabras, enunciados))

for elemento in lista_td_idf:
    elemento.calcular_TF_IDF()

for enunciado, palabras, elemento in zip(enunciados, palabras_por_enunciado, lista_td_idf):
    print(enunciado, "\n", palabras, "\n", elemento.resumen, "\n", f"Relevancia: {elemento.relevancia}\n")

 to mrs saville england 
 ['to', 'mrs', 'saville', 'england'] 
 [('mrs', 0.8460975658364436), ('saville', 0.8460975658364436), ('england', 0.7447312888094024)] 
 Relevancia: 2.4369264204822896

st petersburgh dec th  
 ['st', 'petersburgh', 'dec', 'th'] 
 [('petersburgh', 0.8460975658364436), ('dec', 0.8460975658364436), ('st', 0.17328679513998632), ('th', 0.051584108249457146)] 
 Relevancia: 1.9170660350623308

you will rejoice to hear that no disaster has accompanied the commencement of an enterprise which you have regarded with such evil forebodings 
 ['you', 'will', 'rejoice', 'to', 'hear', 'that', 'no', 'disaster', 'has', 'accompanied', 'the', 'commencement', 'of', 'an', 'enterprise', 'which', 'you', 'have', 'regarded', 'with', 'such', 'evil', 'forebodings'] 
 [('rejoice', 0.17728423669155302), ('disaster', 0.17728423669155302), ('commencement', 0.17728423669155302), ('regarded', 0.17728423669155302), ('forebodings', 0.17728423669155302), ('accompanied', 0.15965531894771978), ('ev

In [7]:
lista_td_idf = sorted(lista_td_idf, key=lambda elemento: elemento.relevancia, reverse=True)
for i in range(0, 5):
    print((" ".join(lista_td_idf[i].tokens)).center(70, " ") + (f"{lista_td_idf[i].relevancia}").center(30, " "))

                                  rw                                        3.672072335797555       
                    heaven bless my beloved sister                          2.7106618820702475      
                 farewell my dear excellent margaret                        2.660398996414066       
    your affectionate brotherrobert walton to mrs saville england           2.646173474678749       
       your affectionate brotherr walton to mrs saville england             2.595490336165228       


### Frecuencia de palabras normalizada
De acuerdo con [Matthew Mayo](https://www.kdnuggets.com/2019/11/getting-started-automated-text-summarization.html) y [Dante Sblendorio](https://www.activestate.com/blog/how-to-do-text-summarization-with-python/). Se debe seguir el siguiente algoritmo:
1. Normalizar los documentos.
2.  Por cada palabra en el corpus se cuenta su frecuencia
3.  Se obtiene la frecuencia más alta
4.  Se divide cada frecuencia entre la frecuencia más alta
5.  Por cada enunciado y cada palabra en el corpus, si la palabra se encuentra en el enunciado, se aumenta el conteo del enunciado
7.  Se ordenan los enunciados por su puntación y se obtiene el resumen

Seguiremos este algoritmo a continuación:

In [8]:
class Frecuencias_normalizadas():
    def __init__(self, corpus: list[str], tokens_por_enunciado: list[list[str]], lang: str = "english"):
        self.corpus: list[str] = corpus
        # Agregamos las palabras de todo el cuerpo
        bolsa_palabras: set[str] = []
        self.stopWords: set = nltk.corpus.stopwords.words(lang) # Definimos el conjunto de stopwords
        for enunciado in tokens_por_enunciado:
            for token in enunciado:
                if token in self.stopWords or len(token) < 3: # Se tuvo que agregar el filtro debido al escape de palabras cortas irrelevantes
                    continue
                else:
                    bolsa_palabras.append(token)
        self.palabras: list[str] = list(bolsa_palabras)
        self.frecuencias_palabras: dict = {}
        self.puntaje_enunciado: dict = {}
        self.resumen: list = []
    
    def calcular_frecuencias(self) -> None:
        self.frecuencias_palabras = {}
        self.puntaje_enunciado = {}
        self.resumen = []
        for enunciado in self.corpus:
            #print (enunciado)
            for token in self.palabras:
                #print(token)
                if token in self.frecuencias_palabras.keys():
                    self.frecuencias_palabras[token] += 1
                else:
                    self.frecuencias_palabras[token] = 1
        
        max_frecuencia: int = max(self.frecuencias_palabras.values())
        
        for palabra in self.frecuencias_palabras.keys():
            self.frecuencias_palabras[palabra] /= max_frecuencia
        
        # Descomente las siguiente líneas para visualizar las palabras por su frecuencia normalizada
        #for key, value in self.frecuencias_palabras.items():
        #    print("Palabra:", f"{key}".center(21, ' '), f".\tFrecuencia: {value:.2f}")

        for enunciado in self.corpus:
            for word in self.palabras:
                if word in enunciado:
                    if enunciado in self.puntaje_enunciado.keys():
                        self.puntaje_enunciado[enunciado] += 1
                    else:
                        self.puntaje_enunciado[enunciado] = 1

        self.resumen = sorted(self.puntaje_enunciado, key=lambda enunciado: self.puntaje_enunciado[enunciado], reverse=True)


In [9]:
normalizadas: Frecuencias_normalizadas = Frecuencias_normalizadas(enunciados, palabras_por_enunciado)
normalizadas.calcular_frecuencias()

print(" Resumen: ".center(100, '-'))
for i, enunciado in enumerate(normalizadas.resumen[0:5]):
    print(f"Enunciado {i+1}:\n{enunciado}")

--------------------------------------------- Resumen: ---------------------------------------------
Enunciado 1:
a youth passed in solitude my best years spent under your gentle and feminine fosterage has so refined the groundwork of my character that i cannot overcome an intense distaste to the usual brutality exercised on board ship i have never believed it to be necessary and when i heard of a mariner equally noted for his kindliness of heart and the respect and obedience paid to him by his crew i felt myself peculiarly fortunate in being able to secure his services
Enunciado 2:
but i have one want which i have never yet been able to satisfy and the absence of the object of which i now feel as a most severe evil i have no friend margaret when i am glowing with the enthusiasm of success there will be none to participate my joy if i am assailed by disappointment no one will endeavour to sustain me in dejection
Enunciado 3:
i accompanied the whale-fishers on several expeditions to the

### RAKE - NLTK
En el ejemplo citado por [Manmohan Singh](https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c):
1. Se importa el algoritmo Rake
2. Se guarda en una variable
3. Se aplica el algoritmo sobre el texto original
4. Se obtiene el resumen
```
>>> from rake_nltk import Rake
>>> rake_nltk_var = Rake()
>>> text = """spaCy is an open-source software library for advanced natural language processing,
written in the programming languages Python and Cython. The library is published under the MIT license
and its main developers are Matthew Honnibal and Ines Montani, the founders of the software company Explosion."""
>>> rake_nltk_var.extract_keywords_from_text(text)
>>> keyword_extracted = rake_nltk_var.get_ranked_phrases()
>>> print(keyword_extracted)
['advanced natural language processing', 'software company explosion', 
 'programming languages python', 'source software library', 'mit license',
 'matthew honnibal', 'main developers', 'ines montani', 'library', 'written',
 'spacy', 'published', 'open', 'founders', 'cython']
```
> Singh, M., "Keyword Extraction process in Python with Natural Language Processing(NLP)", Medium. Recuperado en: [_https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c_](https://towardsdatascience.com/keyword-extraction-process-in-python-with-natural-language-processing-nlp-d769a9069d5c)

Se usará el método integrado en NLTK para extraer las palabaras clave y, posteriormente, obtener el resumen con los enunciados que contengan estas palabras clave

In [16]:
alg_rake: Rake = Rake()
# Para este es necesario juntar los enunciados en un solo texto y lo evaluamos en la función
alg_rake.extract_keywords_from_text(". ".join(enunciados))
palabras_clave = alg_rake.get_ranked_phrases()[:5]
print(f"Frases clave encontradas:\n{palabras_clave}\nFrases completas:")
for palabras in palabras_clave:
    for enunciado in enunciados:
        if palabras in enunciado:
            print(enunciado)


Frases clave encontradas:
['voluntarily endured cold famine thirst', 'succeed many many months perhaps years', 'old man decidedly refused thinking', 'cold northern breeze play upon', 'beauty every region hitherto discovered']
Frases completas:
i accompanied the whale-fishers on several expeditions to the north sea i voluntarily endured cold famine thirst and want of sleep i often worked harder than the common sailors during the day and devoted my nights to the study of mathematics the theory of medicine and those branches of physical science from which a naval adventurer might derive the greatest practical advantage
if i succeed many many months perhaps years will pass before you and i may meet
but the old man decidedly refused thinking himself bound in honour to my friend who when he found the father inexorable quitted his country nor returned until he heard that his former mistress was married according to her inclinations
i am already far north of london and as i walk in the streets